<a href="https://colab.research.google.com/github/Rajkdea123/Face-Mask-Detection/blob/main/Live_Face_Mask_Detection_App_From_Single_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from google.colab.patches import cv2_imshow

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/model/resnet.h5')

# Image dimensions expected by the model
img_width, img_height = 200, 200

# Load the Cascade face classifier
face_cascade = cv2.CascadeClassifier("/content/drive/MyDrive/live mask detection app/haarcascade_frontalface_default.xml")

# Font parameters for text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
color = (255, 0, 0)
thickness = 2

# Load the color image
color_img = cv2.imread('/content/drive/MyDrive/live mask detection app/images/active-attractive-beautiful-1001850.jpg')

# Convert the image to grayscale
gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

# Detect faces in the grayscale image
faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)

# Process each detected face
for i, (x, y, w, h) in enumerate(faces):
    org = (x - 10, y - 10)
    color_face = color_img[y:y+h, x:x+w]  # Extract color face
    face_path = '/content/drive/MyDrive/live mask detection app/images/%dface.jpg' % (i + 1)

    # Save face image
    if cv2.imwrite(face_path, color_face):
        try:
            # Load face image for prediction
            img = load_img(face_path, target_size=(img_width, img_height))
            img = img_to_array(img) / 255
            img = np.expand_dims(img, axis=0)
            pred_prob = model.predict(img)
            pred = np.argmax(pred_prob)

            if pred == 0:
                print("User with mask - prediction =", pred_prob[0][0])
                class_label = "Mask"
                color = (255, 0, 0)
                cv2.imwrite('faces/with_mask/%dface.jpg' % (i + 1), color_face)
                cv2.rectangle(color_img, (x, y), (x + w, y + h), color, thickness)
                cv2.putText(color_img, class_label, org, font, font_scale, color, thickness)
            else:
                print('User not wearing mask - probability =', pred_prob[0][1])
                class_label = "No Mask"
                color = (0, 255, 0)
                cv2.imwrite('faces/without_mask/%dface.jpg' % (i + 1), color_face)
                cv2.rectangle(color_img, (x, y), (x + w, y + h), color, thickness)
                cv2.putText(color_img, class_label, org, font, font_scale, color, thickness)

        except Exception as e:
            print(f"Error processing face {i + 1}: {e}")
    else:
        print(f"Failed to save face {i + 1} image")

# Display the color image with rectangles and labels
cv2_imshow(color_img)
